In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
trainData = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2022/data.csv', index_col = 'row_id')

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from tqdm import tqdm
def reduce_mem_usage(df:pd.DataFrame)->pd.DataFrame:
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in tqdm(df.columns):
        col_type = df[col].dtype
        name =df[col].dtype.name 
        
        if col_type != object and col_type.name != 'category':
        #if name != "category":    
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [5]:
trainData = reduce_mem_usage(trainData)

In [6]:
from collections import defaultdict
featureBuckets = defaultdict(list)

for cols in trainData.columns[1::]:
    featureBuckets[cols[0:3]].append(cols)

for bucket in featureBuckets:
    print(f'{bucket} bucket contains {len(featureBuckets[bucket])} items')
    
features = trainData.copy()

In [7]:
for bucket in featureBuckets:
    print(features[featureBuckets[bucket]].describe().transpose())

In [8]:
def drawCorrelationMatrix(df):
    
    cmap = sns.diverging_palette(250, 15, s=75, l=40,
                                 n=9, center="light", as_cmap=True)
    matrix = df.corr()
    mask = np.triu(np.ones_like(matrix, dtype=bool))
    plt.figure(figsize=(16,12))
    plt.title("Correlation heatmap between all parameters")
#     sns.heatmap(corr, mask=mask, robust=True, center=0,square=True,fmt='.2f', linewidths=.6,cmap='YlOrRd_r')
    fig = sns.heatmap(matrix, robust = True,mask=mask, center=0,linewidths = 0.6, square=True, cmap=cmap)
    
def getRedundantPairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def getTopAbsCorrelations(df, n=5,method = 'pearson'):
#   ''' Get top 5 most correlated features in the given dataset. Method can be 'pearson', 'spearman', 'kendall' ''''
    au_corr = df.corr(method = method).abs().unstack()
    labels_to_drop = getRedundantPairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    print("Top {} Correlated features in given dataset".format(n))
    return au_corr[0:n]

In [9]:
drawCorrelationMatrix(features)

In [10]:
getTopAbsCorrelations(features, n=10,method = 'pearson')

In [ ]:
features[featureBuckets['F_1']].columns

In [11]:
continuousFeatures = [cols for cols in features.columns if features[cols].dtypes=='float64']

cfMean = features[continuousFeatures].mean()
cfMedian = features[continuousFeatures].median()
cfMode = features[continuousFeatures].mode()

In [12]:
F123 = []
for bucket in featureBuckets:
    if bucket is not 'F_4':F123.extend(features[featureBuckets[bucket]].columns)

In [13]:
F123

In [15]:
f4Features = pd.DataFrame()

for f in features[featureBuckets['F_4']].columns:
    f4Features[f] = features[f]

In [ ]:
from numpy import isnan
from sklearn.impute import KNNImputer
from xgboost import XGBRegressor


# finalTransformed = pd.DataFrame(columns = features)
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

# feat_imp = SimpleImputer(missing_values=np.nan, strategy= 'mean')
imp = pd.DataFrame(imputer.fit_transform(features[F123]), columns = F123)


for column in f4Features.columns:
    df_train = f4Features[f4Features[column].isna() == False]
    df_test = f4Features[f4Features[column].isna() == True]
    
    X = df_train.drop(column, axis=1)
    y = df_train[column]
    
    model = XGBRegressor(random_state=123, n_estimators= 3000, booster='gbtree',verbosity = 2,tree_method = 'gpu_hist')
    model.fit(X, y)
    
    pred = model.score(X, y)
    df_new[column][df_test.index] = model.predict(df_test.drop(column, axis = 1))


# train_knn['Age'] = imputer.fit_transform(train_knn[['Age']])
# # fit on the dataset
# imputer.fit(features)
# # transform the dataset
# Xtrans = imputer.transform(features)



In [ ]:
df_subm =  pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv', index_col='row-col')
df = pd.concat([imp, f4Features], axis = 1)
for i in tqdm(df_subm.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    df_subm.loc[i, 'value'] = df.loc[row, col]
df_subm.to_csv('submission_xgb.csv')
df_subm